IMPORTING REQUIRED LIBRARIES

In [11]:
import numpy as np 
import pandas as pd
import math 
import time
import seaborn as sns
import xgboost as xgb
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [12]:
tqdm.pandas()

READING THE DATA

In [ ]:
train = pd.read_csv('data_set/train.csv')
test = pd.read_csv('data_set/test.csv')
sample_submission = pd.read_csv('data_set/sample_submission.csv')

order_id : unique id for each order
order_time: time of the creation of order by the client
order_date : date of the order
allot_time: time of allocation of order to the rider
accept_time: time of acceptance of the order by the rider (if available)
pickup_time: time of pickup of the order (if available)
delivered_time: time of delivery of the order (if available)
cancelled_time: time of cancellation of order (if the order was cancelled)
cancelled: whether the order was cancelled
rider_id: unique id for each rider
first_mile_distance: road distance from rider’s location to the pickup location
last_mile_distance: road distance from pickup location to the delivery location
allotted_orders: total number of orders allotted to the rider in the 30 days before (not including) order_date
delivered_orders: total number of orders delivered by the rider in the 30 days before (not including) order_date
undelivered_orders: total number of orders allotted to but not delivered by the rider (i.e. cancelled) in the 30 days before (not including) order_date
lifetime_order_count: total number of orders delivered by the rider at any time before order_date
reassigned_order: whether the order was reassigned to this rider
reassignment_method: if the order was reassigned, whether the reassignment was done manually (by the ops team) or automatically
reassignment_reason: a more detailed reason for the reassignment
session_time: total time the rider had been online on order_date before order_time

In [19]:
id_columns = ['order_id']

time_columns = ['order_time', 'order_date', 'allot_time', 'accept_time',
       'pickup_time', 'delivered_time','cancelled_time']

categorical_columns = ['reassignment_method','reassignment_reason']

CONVERTING TO DATETIME AND OBJECT


In [20]:
ids = train[id_columns]
train = train.drop(ids, axis=1)
test = test.drop(ids,axis=1)

for col in time_columns:
    train[col] = pd.to_datetime(train[col])
    try:
        test[col] = pd.to_datetime(test[col])
    except:
        continue

for col in categorical_columns:
    train[col] = train[col].astype('object')
    try:
        test[col] = test[col].astype('object')
    except:
        continue

C:\Users\Nikhil\AppData\Local\Temp\ipykernel_2624\338228778.py:6: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train[col] = pd.to_datetime(train[col])
C:\Users\Nikhil\AppData\Local\Temp\ipykernel_2624\338228778.py:6: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train[col] = pd.to_datetime(train[col])
C:\Users\Nikhil\AppData\Local\Temp\ipykernel_2624\338228778.py:6: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train[col] = pd.to_datetime(train[col])
C:\Users\Nikhil\AppData\Local\Temp\ipykernel_2624\338228778.py:6: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `da